In [1]:
import torch.nn as nn
import torch.optim as optim
import torch
import mlflow
import mlflow.pytorch
from sklearn.metrics import precision_score, recall_score, f1_score
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
from PIL import Image
from zipfile import ZipFile
from tqdm import tqdm

In [18]:
# === MODEL DEFINITION ===
def create_cnn_model(in_channels=3, conv_filters=[32, 64], kernel_size=3, fc_units=[128], 
                     dropout=0.5, num_classes=3, flatten_type="flatten"):
    layers = []
    input_c = in_channels

    for out_c in conv_filters:
        layers.append(nn.Conv2d(input_c, out_c, kernel_size, padding=1))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(2))
        input_c = out_c

    if flatten_type == "gap":
        layers.append(nn.AdaptiveAvgPool2d((1, 1)))
        layers.append(nn.Flatten())
        num_flat_features = conv_filters[-1] * 1 * 1
    else:
        layers.append(nn.AdaptiveAvgPool2d((4, 4)))
        layers.append(nn.Flatten())
        num_flat_features = conv_filters[-1] * 4 * 4

    for units in fc_units:
        layers.append(nn.Linear(num_flat_features, units))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        num_flat_features = units

    layers.append(nn.Linear(num_flat_features, num_classes))
    return nn.Sequential(*layers)

In [23]:
model_configs = [
    {
        "in_channels": 3,
        "conv_filters": [32, 64],
        "fc_units": [256],
        "dropout": 0.5,
        "flatten_type": "flatten",
        "num_classes": 3
    },
    {
        "in_channels": 3,
        "conv_filters": [64, 128],
        "fc_units": [512, 128],
        "dropout": 0.4,
        "flatten_type": "gap",
        "num_classes": 3
    },
    {
        "in_channels": 3,
        "conv_filters": [32, 64, 128],
        "fc_units": [1024, 512, 128],
        "dropout": 0.3,
        "flatten_type": "flatten",
        "num_classes": 3
    }
]


In [3]:
def get_loss_function(loss_name='bcewithlogits'):
    if loss_name == 'bcewithlogits':
        return nn.BCEWithLogitsLoss()
    elif loss_name == 'bceloss':
        return nn.BCELoss()
    elif loss_name == 'mse':
        return nn.MSELoss()
    else:
        raise ValueError(f"Unsupported loss function: {loss_name}")

In [4]:
def get_optimizer(optimizer_name, model_parameters, lr=1e-3, weight_decay=0):
    if optimizer_name.lower() == 'adam':
        return optim.Adam(model_parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_name.lower() == 'sgd':
        return optim.SGD(model_parameters, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name.lower() == 'rmsprop':
        return optim.RMSprop(model_parameters, lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

In [7]:
# Load tensors
train_tensors = torch.load('train_tensors.pt')
val_tensors = torch.load('val_tensors.pt')
test_tensors = torch.load('test_tensors.pt')

C:\Users\Rahhal AbuZahra\AppData\Local\Temp\ipykernel_19696\2242991132.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_tensors = torch.load('train_tensors.pt')
C:\

In [8]:
train_dataset = TensorDataset(*train_tensors)
val_dataset = TensorDataset(*val_tensors)
test_dataset = TensorDataset(*test_tensors)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # adjust batch_size if needed
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [19]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    best_val_acc = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.numel()

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                probs = torch.sigmoid(outputs)
                preds = (probs > 0.5).float()
                val_correct += (preds == labels).sum().item()
                val_total += labels.numel()

                all_preds.append(preds.cpu())
                all_labels.append(labels.cpu())

        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()

        val_acc = val_correct / val_total
        val_loss /= val_total

        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

        print(f"Epoch {epoch+1} - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, F1: {f1:.4f}")

        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "val_precision": precision,
            "val_recall": recall,
            "val_f1": f1
        }, step=epoch)

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return model, best_val_acc

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
mlflow.set_experiment("celebA_models")

2025/05/18 23:02:32 INFO mlflow.tracking.fluent: Experiment with name 'celebA_models' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/4th%20year/2nd%20semester/Training%203/Final%20Project/mlruns/555883558464305941', creation_time=1747598552165, experiment_id='555883558464305941', last_update_time=1747598552165, lifecycle_stage='active', name='celebA_models', tags={}>

In [26]:
for i, config in enumerate(model_configs):
    
    with mlflow.start_run(run_name=f"celeba_model_{i+1}"):
        config["in_channels"] = 3
        
        mlflow.log_params(config)

        model = create_cnn_model(**config)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        trained_model, best_val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device)

        mlflow.log_metric("best_val_acc", best_val_acc)

        # Save model artifact
        model_path = f"model_{i+1}.pt"
        torch.save(trained_model.state_dict(), model_path)
        mlflow.log_artifact(model_path)
        os.remove(model_path)

        mlflow.end_run()

Epoch 1/10 [Val]: 100%|██████████| 79/79 [00:43<00:00,  1.81it/s]


Epoch 1 - Train Acc: 0.6473, Val Acc: 0.7086, F1: 0.5802


Epoch 2/10 [Val]: 100%|██████████| 79/79 [00:39<00:00,  1.99it/s]


Epoch 2 - Train Acc: 0.7185, Val Acc: 0.7769, F1: 0.6237


Epoch 3/10 [Val]: 100%|██████████| 79/79 [00:39<00:00,  2.00it/s]


...

Epoch 8 - Train Acc: 0.8795, Val Acc: 0.8928, F1: 0.7717


Epoch 9/10 [Val]: 100%|██████████| 79/79 [00:49<00:00,  1.58it/s]


Epoch 9 - Train Acc: 0.8952, Val Acc: 0.9071, F1: 0.8067


Epoch 10/10 [Val]: 100%|██████████| 79/79 [00:50<00:00,  1.57it/s]


Epoch 10 - Train Acc: 0.9050, Val Acc: 0.9183, F1: 0.8285


In [2]:
from torchvision import models

In [28]:
# Step 1: Create ResNet Model 
def create_resnet_for_multilabel(num_labels=3, freeze_features=False):
    model = models.resnet18(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, num_labels),
        nn.Sigmoid()
    )
    if freeze_features:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.fc.parameters():
            param.requires_grad = True
    return model

In [31]:
# Step 3: Evaluation Function 
def evaluate_model(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for imgs, labels in data_loader:
            imgs, labels = imgs.to(device), labels.to(device).float()
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item() * imgs.size(0)

            preds = (outputs > 0.5).float()
            total_correct += (preds == labels).sum().item()
            total_samples += labels.numel()

    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy


In [36]:
# Step 2: Training Loop 
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device):
    
    with mlflow.start_run(run_name='resnet_model_run'):
        model.to(device)
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for imgs, labels in train_loader:
                imgs, labels = imgs.to(device), labels.to(device).float()

                optimizer.zero_grad()
                outputs = model(imgs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * imgs.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)

            # Validation
            model.eval()
            val_loss, val_acc = evaluate_model(model, val_loader, loss_fn, device)

            print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            mlflow.log_metric("train_loss", epoch_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            mlflow.log_metric("val_accuracy", val_acc, step=epoch)

        # Log model and parameters
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("optimizer", type(optimizer).__name__)
        mlflow.log_param("learning_rate", optimizer.param_groups[0]['lr'])
        mlflow.pytorch.log_model(model, "model")

In [37]:
model = create_resnet_for_multilabel(num_labels=3)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_model(model , train_loader , val_loader , loss_fn , optimizer , 10 , device)

with open("resnet_model.pkl", "wb") as f:
    torch.save(model, f)

torch.save(model.state_dict(), "resnet_model_weights.pkl")

c:\Users\Rahhal AbuZahra\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Rahhal AbuZahra\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] Loss: 0.5842 | Val Loss: 0.6023 | Val Acc: 0.9374
Epoch [2/10] Loss: 0.5301 | Val Loss: 0.5979 | Val Acc: 0.9395
Epoch [3/10] Loss: 0.5204 | Val Loss: 0.5972 | Val Acc: 0.9481
Epoch [4/10] Loss: 0.5155 | Val Loss: 0.5972 | Val Acc: 0.9499
Epoch [5/10] Loss: 0.5128 | Val Loss: 0.5982 | Val Acc: 0.9520
Epoch [6/10] Loss: 0.5111 | Val Loss: 0.5953 | Val Acc: 0.9515
Epoch [7/10] Loss: 0.5102 | Val Loss: 0.6010 | Val Acc: 0.9488
Epoch [8/10] Loss: 0.5094 | Val Loss: 0.5938 | Val Acc: 0.9531
Epoch [9/10] Loss: 0.5081 | Val Loss: 0.5946 | Val Acc: 0.9541
Epoch [10/10] Loss: 0.5081 | Val Loss: 0.5934 | Val Acc: 0.9545


2025/05/19 02:12:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
def evaluate_model_on_test(model, test_loader, threshold=0.5, device='cuda'):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.cpu().numpy()
            outputs = model(imgs).cpu().numpy()
            preds = (outputs > threshold).astype(int)

            all_labels.extend(labels)
            all_preds.extend(preds)

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    accuracy = (all_labels == all_preds).mean()
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    print("==== Test Evaluation ====")
    print(f"Exact Match Accuracy: {accuracy:.4f}")
    print(f"Macro F1 Score:       {f1:.4f}")
    print(f"Macro Precision:      {precision:.4f}")
    print(f"Macro Recall:         {recall:.4f}")

    return accuracy, f1, precision, recall

In [39]:
# Evaluate model on test set
accuracy, f1, precision, recall = evaluate_model_on_test(model, test_loader, threshold=0.5, device=device)

# log to MLflow
mlflow.log_metric("test_accuracy", accuracy)
mlflow.log_metric("test_f1", f1)
mlflow.log_metric("test_precision", precision)
mlflow.log_metric("test_recall", recall)


==== Test Evaluation ====
Exact Match Accuracy: 0.9522
Macro F1 Score:       0.8951
Macro Precision:      0.8815
Macro Recall:         0.9205


In [3]:
# Step 4: Test Prediction on One Image 
def predict_on_image(model, image_tensor, device):
    model.eval()
    image_tensor = image_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        preds = (output > 0.5).int().cpu().numpy()[0]
    return preds

In [4]:
from torchvision import transforms


def create_transforms():
    """Create preprocessing transform pipelines for train and validation/test"""

    # Training transforms with augmentation
    train_transform = transforms.Compose([
        transforms.Resize((178, 178)),
        transforms.RandomCrop((160, 160)), # Removes some background and focuses on the face
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),# Slightly vary brightness/contrast
        transforms.ToTensor(), # Convert to tensor and scale to [0,1]
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize using ImageNet statistics
    ])

    # Validation/Test transforms without augmentation
    val_test_transform = transforms.Compose([
        transforms.Resize((178, 178)),
        transforms.CenterCrop((160, 160)), # Center crop (consistent, no randomness)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_test_transform

In [5]:
import zipfile
from io import BytesIO

def load_image_from_zip(image_filename, zip_file):
    """Load a single image from an already opened zip file"""
    image_path_in_zip = f"img_align_celeba/img_align_celeba/{image_filename}"

    try:
        # Read the image data into BytesIO first to make it seekable
        with zip_file.open(image_path_in_zip) as image_data:
            # Read all data into memory and create a seekable BytesIO object
            image_bytes = BytesIO(image_data.read())

            # Convert to PIL Image using the seekable BytesIO object
            image = Image.open(image_bytes)

            # Convert to RGB if needed
            if image.mode != 'RGB':
                image = image.convert('RGB')

            return image
    except Exception as e:
        print(f"error loading image {image_filename}: {e}")
        # Return a blank image as fallback
        return Image.new('RGB', (178, 178), color='black')

In [17]:
image_path='000067.jpg'
zip_path='Data/img_align_celeba.zip'

In [8]:
train_transform , val_transform=create_transforms()

In [18]:
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    image = load_image_from_zip(image_path, zip_file)  
    image = val_transform(image)

In [11]:
with open("resnet_model.pkl", "rb") as f:
    model = torch.load(f)
model.eval()

C:\Users\Rahhal AbuZahra\AppData\Local\Temp\ipykernel_18540\4021340549.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
predictions = predict_on_image(model, image, device)

In [15]:
predictions

array([0, 0, 1])

In [20]:
# 5. Map predictions to attribute names
# attribute_names = ['Smiling','Male','Wearing Hat']
# predicted_attributes = [attr for attr, pred in zip(attribute_names, predictions) if pred == 1]

# 6. Print results

print({
    "Smiling": "Smiling" if predictions[0] == 1 else "Not Smiling",
    "Gender": "Male" if predictions[1] == 1 else "Female",
    "Hat": "Wearing Hat" if predictions[2] == 1 else "Not Wearing Hat"
})

{'Smiling': 'Not Smiling', 'Gender': 'Male', 'Hat': 'Not Wearing Hat'}
